# Gaia Density Computations Example

This document contains an example of the Gaia Density Computations plugin.

The original Python notebook is available in the plugin source code under the docs/examples folder.

In [1]:
# Import required modules
import folium
import numpy as np
from folium.plugins import ImageOverlay
from gaia_densitycomputations.processes import SimpleGridDensityProcess, KernelDensityProcess
from gaia.geo.geo_inputs import VectorFileIO

## Simple Grid Density Process

Create a SimpleGridDensityProcess instance to determine number of points within each cell of a 200 column x 100 row grid, covering the same geographic extent as the input layer.

In [2]:
# Point dataset to calculate density of
ports = VectorFileIO(uri='../../tests/data/ports_and_harbours.geojson')

# Grid width to use for density calculation
width=200

# Create an instance of the Simple Density Process
sdp = SimpleGridDensityProcess(inputs = [ports], width=width)

Compute the least cost path

In [7]:
sdp.compute()
print sdp.output.uri

/Users/mbertrand/dev/kitware/gaia/tests/data/output/c5fff097-ad3b-48e8-89c7-e641d6240e63/c5fff097-ad3b-48e8-89c7-e641d6240e63.tif


Display the density on a map

In [4]:
world_map = folium.Map([50,-90],
                  zoom_start=4,
                  tiles='cartodbpositron')

temparray = np.array(sdp.output.read().GetRasterBand(1).ReadAsArray())
overlay = ImageOverlay(temparray, 
                       [[-90.0,-180.0],[90.0,180.0]], 
                       colormap=lambda x: (1, 0, 0, x),
                       mercator_project=True, control=True)
overlay.layer_name = 'Port Density'

world_map.add_children(overlay)

folium.LayerControl().add_to(world_map)
world_map

## Kernel Density Process

In [5]:
# Calculate kernel density with default bandwidth estimator ('scott')
inputio = VectorFileIO(uri='/Users/mbertrand/dev/kitware/gaia-densitycomputations-plugin/tests/data/iraq_hospitals.geojson')
kdp = KernelDensityProcess(inputs=[inputio], width=200)
kdp.compute()
print kdp.output.uri

/Users/mbertrand/dev/kitware/gaia/tests/data/output/4b84bd47-12c2-4e96-9340-7b7cfb1a2ecc/4b84bd47-12c2-4e96-9340-7b7cfb1a2ecc.tif


In [6]:
# Display kernel density on map
from folium.plugins import MarkerCluster

world_map = folium.Map([34.00,43.00],
                  zoom_start=6,
                  tiles='cartodbpositron')

hospitals = folium.features.GeoJson(inputio.read().to_json(), name='Hospitals')
temparray = np.array(kdp.output.read().GetRasterBand(1).ReadAsArray())
overlay = ImageOverlay(temparray, [[27.0466603,38.9358608],[40.3888093,50.8226656]], 
                       colormap=lambda x: (1, 0, 0, x),
                       mercator_project=True, control=True)
overlay.layer_name = 'Kernel Density'

world_map.add_children(overlay)
world_map.add_children(hospitals)

folium.LayerControl().add_to(world_map)
world_map

In [8]:
# Calculate kernel density with a custom bandwidth value
inputio = VectorFileIO(uri='/Users/mbertrand/dev/kitware/gaia-densitycomputations-plugin/tests/data/iraq_hospitals.geojson')
kdp = KernelDensityProcess(inputs=[inputio], width=200, bandwidth=0.8)
kdp.compute()
print kdp.output.uri

/Users/mbertrand/dev/kitware/gaia/tests/data/output/09c8a41b-71c7-4805-9f18-e084c6774933/09c8a41b-71c7-4805-9f18-e084c6774933.tif


In [9]:
# Display kernel density on map
from folium.plugins import MarkerCluster

world_map = folium.Map([34.00,43.00],
                  zoom_start=6,
                  tiles='cartodbpositron')

hospitals = folium.features.GeoJson(inputio.read().to_json(), name='Hospitals')
temparray = np.array(kdp.output.read().GetRasterBand(1).ReadAsArray())
overlay = ImageOverlay(temparray, [[27.0466603,38.9358608],[40.3888093,50.8226656]], 
                       colormap=lambda x: (1, 0, 0, x),
                       mercator_project=True, control=True)
overlay.layer_name = 'Kernel Density'

world_map.add_children(overlay)
world_map.add_children(hospitals)

folium.LayerControl().add_to(world_map)
world_map